In [ ]:
#pip install pandas

In [ ]:
#pip install seaborn

In [ ]:
#pip install numpy==1.20.3

In [ ]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns

import sys
sys.path.append("../")
from packages.plotter import *
import warnings
warnings.simplefilter("ignore")

In [ ]:
df_expanded = pd.read_pickle("pickles/spectrum_green_matrix.pkl")
radius = 0.0001
fs = 16

In [ ]:
def plot_configuration_sweep(multiplier, numberOfBubbles, scatter, xlims=None, ylims=None, title=None, single=False):
    with open('pickles/L={}*radius/2${}'.format(multiplier, numberOfBubbles**2), 'rb') as file:
        pickleDict = pickle.load(file)
    
    wavenumberSample = np.array(pickleDict['wavenumbers'])
    print(len(set(wavenumberSample)))
    kaSample = [k*radius for k in wavenumberSample]
    totalFields = pickleDict['totalFields']
    absoluteTotalFields, realTotalFields, imaginaryTotalFields = getAbsoluteRealImaginaryValues(totalFields)
    absoluteTotalFields = [totalField for totalField in absoluteTotalFields]
    
    while kaSample[0] < 0.0027 or kaSample[-1] > 0.02:
        for i in range(len(kaSample)):
            if kaSample[i] < 0.0027 or kaSample[i] > 0.02:
                kaSample.pop(i)
                absoluteTotalFields.pop(i)
                break
    
    fig = plt.figure(figsize=(15, 5))
    sns.set_theme(style="darkgrid")
    ax = fig.add_subplot()
    
    relevant_df = df_expanded[(df_expanded.distance == multiplier) & (df_expanded.configuration == numberOfBubbles) & (df_expanded.excitation_factor_normalized > 0.01)]
    relevant_df.sort_values("excitation_factor_normalized", ascending=False, inplace=True)
        
    if single:
        minnaert = list(df_expanded.wavenumber)[0]
        plt.axvline(x=minnaert, color="g", linewidth=2, label="single bubble")

    wavenumbers = list(relevant_df.wavenumber_resonance)
    exitation_factors = list(relevant_df.excitation_factor_normalized)
    quality_factors = list(df_expanded.qfactor_resonance)
    
    label = True
    for i in range(len(wavenumbers)):
        k = wavenumbers[i]
        exitation_factor = exitation_factors[i]
        quality_factor = quality_factors[i]
        
        print(round(k,5), round(k/radius))
        if exitation_factor < 0.2:
            width = 0.3
        elif exitation_factor < 0.4:
            width = 1
        elif exitation_factor < 0.6:
            width = 1.7
        elif exitation_factor < 0.8:
            width = 2.4
        else:
            width = 3.1
            
        if label and not scatter:
            plt.axvline(x=k, color="k", linewidth=width, label="resonance mode\n(mass-spring)")
            label = False
        elif not label and not scatter:
            plt.axvline(x=k, color="k", linewidth=width)
    
    if scatter:
        plt.scatter(kaSample, absoluteTotalFields, color="C4")
    
    plt.plot(kaSample, absoluteTotalFields,
             linewidth=3,
             color="C4", label="acoustic field\n(BEM)")

    plt.legend(loc='upper right', fontsize=fs)

    if xlims is None:
        plt.xlim((0.0019,0.0205))
    else:
        plt.xlim(xlims)
    if xlims is None:
        plt.ylim((None,None))
    else:
        plt.ylim(ylims)
    ax.set_xlabel("nondimensional wavenumber")
    ax.set_ylabel("acoustic pressure")
    
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(fs)

    if title is not None:
        if not isinstance(title, str):
            title = str(numberOfBubbles)+" x "+str(numberOfBubbles)+" array\nbubble separation: "+str(multiplier)
        ax.annotate(title,
                    xy=(0.072, 0.795), xycoords='figure fraction',
                    bbox=dict(boxstyle="square", facecolor="0.8", edgecolor='k'),
                    fontsize=fs,
                   )

    plt.savefig('sweep_'+str(multiplier)+'_'+str(numberOfBubbles)+'.pdf', bbox_inches='tight')

The following cell visualizes the frequency response of a rectangular bubble array. First, the acoustic response needs to be simulated with the `generator.ipynb` notebook. Specify the number of bubbles per side (`numberOfBubbles`) and the bubble separation (`multiplier`).

In [ ]:
plot_configuration_sweep(multiplier=8, numberOfBubbles=10, scatter=False, xlims=(0.0035, 0.0175), ylims=(0,None), title=True, single=True)